In [ ]:
import torch
from easy_yolo_ocr.yolov5.models.yolo import Model


cfg_path = "easy_yolo_ocr/yolov5/models/yolov5s.yaml"

model = Model(cfg=cfg_path, ch=3, nc=4)

weights = torch.load("easy_yolo_ocr/yolov5/runs/train/exp12/weights/best.pt", map_location='cpu', weights_only=False)

if 'model' in weights:

    state_dict = weights['model'].float().state_dict()

model.load_state_dict(state_dict)

model.eval()

Overriding model.yaml nc=80 with nc=4

                 from  n    params  module                                  arguments                     


  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  3    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]              
  8                -1  1   1182720  models.common.C3                        [512, 512, 1]                 
  9                -1  1    656896  m

Model(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3(
      (cv1): Conv(
        (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv3): C

In [12]:
import cv2
from easy_yolo_ocr.yolov5.utils.general import non_max_suppression
import pandas as pd


def to_model(img, model):
  img_tensor = torch.from_numpy(img).float() / 255.0  # [H, W, C], нормализация
  img_tensor = img_tensor.permute(2, 0, 1).unsqueeze(0)  # [1, 3, H, W]
  model.eval()
  with torch.no_grad():
    pred = model(img_tensor)[0]
    pred = non_max_suppression(pred, conf_thres=0.4, iou_thres=0.45)
  return [{ name:value for value, name in zip(det, ('x','y','x2','y2','p','c'))} for det in pred[0]]


# 805 / 815
image_path = 'data/image805.jpg'
img = cv2.imread(image_path)  # загружаем в BGR (OpenCV-формат)
sub_images = []
array = []

for detect_obj in to_model(img, model):
  x, y,  x2, y2, p, class_id = detect_obj.values()
  if p < 0.7:
    continue
  sub_images.append(img[ int(y):int(y2) , int(x):int(x2)])
  array.append(detect_obj)


array.sort(key = lambda x: int(x['x']) )
df = pd.DataFrame(array)
df = pd.concat([ df[['x','y','x2','y2','c']].astype(int), df['p'].astype(float).round(2) ], axis=1)

In [13]:
df

,x,y,x2,y2,c,p
0,53,546,64,568,2,0.79
1,53,521,64,542,2,0.75
2,477,603,559,629,3,0.87


In [14]:
cls_names = {0 : 'name', 1:'price', 2:'cnt', 3 : 'total'}

In [15]:
import pandas as pd
import numpy as np
from collections import defaultdict

def group_receipt_items(df, y_tolerance=5):
    """
    Группирует прямоугольники сущностей чека по строкам.

    Параметры:
    - df: DataFrame с колонками ['id', 'x1', 'y1', 'x2', 'y2'] (координаты прямоугольников)
    - y_tolerance: допустимое отклонение по оси Y для объединения в строку (в пикселях)

    Возвращает:
    - Список групп (каждая группа - это строка чека), где каждая группа содержит индексы df
    """

    # Вычисляем вертикальные центры каждого прямоугольника
    df['y_center'] = (df['y'] + df['y2']) / 2

    # Сортируем все прямоугольники по вертикали (сверху вниз)
    df_sorted = df.sort_values('y_center').reset_index(drop=True)

    # Группируем по строкам (если вертикальные центры близки в пределах y_tolerance)
    groups = []
    current_group = [0]  # начинаем с первого элемента

    for i in range(1, len(df_sorted)):
        # Если текущий прямоугольник близок к предыдущему по вертикали
        if abs(df_sorted.loc[i, 'y_center'] - df_sorted.loc[i-1, 'y_center']) <= y_tolerance:
            current_group.append(i)
        else:
            # Добавляем завершенную группу и начинаем новую
            groups.append(current_group)
            current_group = [i]

    # Добавляем последнюю группу
    if current_group:
        groups.append(current_group)

    # Сортируем элементы внутри каждой группы по горизонтали (слева направо)
    for i in range(len(groups)):
        group_indices = groups[i]
        # Сортируем по x1 (левый край прямоугольника)
        sorted_indices = df_sorted.loc[group_indices].sort_values('x').index.tolist()
        groups[i] = sorted_indices

    # Возвращаем оригинальные индексы (из исходного df)
    result = []
    for group in groups:
        original_indices = df_sorted.loc[group, 'index'].tolist()
        result.append(original_indices)

    return result

In [16]:
groups = group_receipt_items(df.reset_index())

print(groups)

[[1], [0], [2]]


In [17]:
sub_images[2]

array([[[107, 114, 109],
        [109, 116, 111],
        [108, 115, 110],
        [ 96, 103,  98],
        [108, 115, 110],
        [100, 107, 102],
        [ 67,  74,  69],
        [ 79,  86,  81],
        [110, 117, 112],
        [110, 117, 112],
        [117, 127, 121]],

       [[105, 112, 107],
        [109, 116, 111],
        [110, 117, 112],
        [111, 118, 113],
        [120, 127, 122],
        [ 83,  90,  85],
        [ 41,  48,  43],
        [ 78,  85,  80],
        [114, 121, 116],
        [103, 110, 105],
        [115, 125, 119]],

       [[107, 114, 109],
        [103, 110, 105],
        [ 98, 105, 100],
        [103, 110, 105],
        [ 82,  89,  84],
        [ 40,  47,  42],
        [ 19,  26,  21],
        [ 71,  78,  73],
        [117, 124, 119],
        [106, 113, 108],
        [108, 118, 112]],

       [[100, 107, 102],
        [ 74,  81,  76],
        [ 53,  60,  55],
        [ 42,  49,  44],
        [  6,  13,   8],
        [  0,   7,   2],
        [ 15,  22, 

In [ ]:
sub_images[3]

IndexError: list index out of range

In [ ]:
sub_images[7]

array([[[154, 162, 155],
        [151, 159, 152],
        [144, 152, 145],
        ...,
        [160, 164, 159],
        [158, 165, 158],
        [157, 166, 156]],

       [[167, 175, 168],
        [149, 157, 150],
        [173, 181, 174],
        ...,
        [151, 155, 150],
        [163, 167, 161],
        [160, 168, 158]],

       [[170, 181, 173],
        [140, 151, 143],
        [131, 139, 132],
        ...,
        [150, 154, 149],
        [163, 167, 161],
        [154, 159, 150]],

       ...,

       [[145, 147, 141],
        [172, 174, 168],
        [162, 164, 158],
        ...,
        [ 94,  98,  93],
        [ 68,  75,  68],
        [114, 123, 113]],

       [[153, 153, 147],
        [168, 168, 162],
        [179, 179, 173],
        ...,
        [ 92,  96,  91],
        [110, 117, 110],
        [108, 119, 109]],

       [[169, 170, 161],
        [163, 163, 157],
        [151, 151, 145],
        ...,
        [168, 172, 167],
        [158, 165, 158],
        [149, 158, 148]]], dtype=uint8)

In [ ]:

df.iloc[groups[1]]

,x,y,x2,y2,c,p
2,146,367,154,378,2,0.56
3,177,369,189,380,2,0.44
7,388,365,433,379,1,0.90


In [ ]:
%ls

drive/        image820.jpg  image841.jpg  image862.jpg  image883.jpg
image800.jpg  image821.jpg  image842.jpg  image863.jpg  image884.jpg
image801.jpg  image822.jpg  image843.jpg  image864.jpg  image885.jpg
image802.jpg  image823.jpg  image844.jpg  image865.jpg  image886.jpg
image803.jpg  image824.jpg  image845.jpg  image866.jpg  image887.jpg
image804.jpg  image825.jpg  image846.jpg  image867.jpg  image888.jpg
image805.jpg  image826.jpg  image847.jpg  image868.jpg  image889.jpg
image806.jpg  image827.jpg  image848.jpg  image869.jpg  image890.jpg
image807.jpg  image828.jpg  image849.jpg  image870.jpg  image891.jpg
image808.jpg  image829.jpg  image850.jpg  image871.jpg  image892.jpg
image809.jpg  image830.jpg  image851.jpg  image872.jpg  image893.jpg
image810.jpg  image831.jpg  image852.jpg  image873.jpg  image894.jpg
image811.jpg  image832.jpg  image853.jpg  image874.jpg  image895.jpg
image812.jpg  image833.jpg  image854.jpg  image875.jpg  image896.jpg
image813.jpg  image834.jpg  image8

In [ ]:
%cd ..

/content/drive/MyDrive/ИТМО/[ITMO] ML subject/Group Project v2/Example-of-solving-a-CV-problem


In [ ]:
%cd dataset/custom_data

/content/drive/MyDrive/ИТМО/[ITMO] ML subject/Group Project v2/Example-of-solving-a-CV-problem/dataset/custom_data


In [ ]:
%cp image8*.jpg "/content/"